In [57]:
import sys
import logging

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import torch
import torch.nn.functional as F

from lrann.datasets import DataLoader, random_train_test_split
from lrann.estimators import ImplicitEst
from lrann.models import BilinearNet, DeepNet, ResNet
from lrann.evaluations import mrr_score, precision_recall_score
from lrann.utils import is_cuda_available

In [46]:
data = DataLoader().load_movielens('100k')

In [47]:
train, test = random_train_test_split(data)

## Estimator using a low-rank approximation model

In [54]:
lra_model = BilinearNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)
lra_est = ImplicitEst(model=lra_model, 
                      n_iter=20, 
                      use_cuda=is_cuda_available())
lra_est.fit(train, verbose=True)

Epoch 0: loss 0.2725904422993705
Epoch 1: loss 0.14597288496709662
Epoch 2: loss 0.12268067814303464
Epoch 3: loss 0.10430571385519062
Epoch 4: loss 0.09362372836295851
Epoch 5: loss 0.08439306648545522
Epoch 6: loss 0.07775582379046031
Epoch 7: loss 0.07610465778944137
Epoch 8: loss 0.07058730716455473
Epoch 9: loss 0.06850218000377509
Epoch 10: loss 0.06607056039248234
Epoch 11: loss 0.06586962781710737
Epoch 12: loss 0.06394558241398984
Epoch 13: loss 0.0629675068682954
Epoch 14: loss 0.06010595720007472
Epoch 15: loss 0.06028020984187675
Epoch 16: loss 0.059038570049630625
Epoch 17: loss 0.05847483069271369
Epoch 18: loss 0.0569427542199518
Epoch 19: loss 0.05626072020383578


In [55]:
prec, recall = precision_recall_score(lra_est, test)
prec.mean(), recall.mean()

(0.08226600985221674, 0.055270432630070136)

In [56]:
mrr_score(lra_est, train).mean(), mrr_score(lra_est, test).mean()

(0.03597764718689598, 0.024920279787758952)

## Estimator using a deep neural model

In [51]:
nn_model = DeepNet(data.n_users, data.n_items, embedding_dim=8, sparse=False, activation=torch.tanh)
nn_est = ImplicitEst(model=nn_model, 
                     n_iter=20, 
                     use_cuda=is_cuda_available())
nn_est.fit(train, verbose=True)

Epoch 0: loss 0.23586470484969732
Epoch 1: loss 0.1894077275379335
Epoch 2: loss 0.18241314966638947
Epoch 3: loss 0.17853012318514797
Epoch 4: loss 0.1750466843886912
Epoch 5: loss 0.17116528122483646
Epoch 6: loss 0.17341163586795613
Epoch 7: loss 0.17019281759503904
Epoch 8: loss 0.1687646459923871
Epoch 9: loss 0.16890244743242128
Epoch 10: loss 0.16608317680479795
Epoch 11: loss 0.1675762428581809
Epoch 12: loss 0.1658183973630899
Epoch 13: loss 0.16381410580193373
Epoch 14: loss 0.16470830679447546
Epoch 15: loss 0.1637588477503101
Epoch 16: loss 0.163464301557391
Epoch 17: loss 0.16170240344742398
Epoch 18: loss 0.1607877084079144
Epoch 19: loss 0.1616262861316442


In [52]:
prec, recall = precision_recall_score(nn_est, test)
prec.mean(), recall.mean()

(0.07536945812807883, 0.0450172280291891)

In [53]:
mrr_score(nn_est, train).mean(), mrr_score(nn_est, test).mean()

(0.019041247839855807, 0.018543117584467632)

## Estimator using a deep residual model

In [ ]:
res_model = ResNet(data.n_users, data.n_items, embedding_dim=32, sparse=False, activation=torch.sigmoid)
res_est = ImplicitEst(model=res_model, 
                     n_iter=20, 
                     use_cuda=is_cuda_available())
res_est.fit(train, verbose=True)

Epoch 0: loss 0.26957950572204287
Epoch 1: loss 0.1518804550572356
Epoch 2: loss 0.1221214220896009
Epoch 3: loss 0.10432872754151205
Epoch 4: loss 0.09417445225886138
Epoch 5: loss 0.08533422598429319
Epoch 6: loss 0.08263045440762999
Epoch 7: loss 0.07659786217587686
Epoch 8: loss 0.07287330987749595
Epoch 9: loss 0.06978465359593061
Epoch 10: loss 0.06837380008414837
Epoch 11: loss 0.0662923973325268
Epoch 12: loss 0.0653633287814099
Epoch 13: loss 0.06402774985655316
Epoch 14: loss 0.06219431265255823
Epoch 15: loss 0.060135974839518644
Epoch 16: loss 0.06133887371367136
Epoch 17: loss 0.06003553657489939
Epoch 18: loss 0.059792405610576436


In [68]:
prec, recall = precision_recall_score(res_est, test)
prec.mean(), recall.mean()

(0.08275862068965517, 0.047558592438274205)

In [69]:
mrr_score(res_est, train).mean(), mrr_score(res_est, test).mean()

(0.028651481479257234, 0.024004779585839523)